# Mike Babb
# babbm@uw.edu
# Introduction to Python Part 02

In [ ]:
# Problem Definition: find all valid words from a given word that uses all 
# letters in the given word. Each word group should contain at least as many words as
# there are letters in the word. For example, the word 'time' is four characters long and those
# characters can be rearranged as follows: emit, item, mite, and time. 

In [ ]:
# standard libraries - installed by default
import csv
from itertools import permutations, combinations
import math
import os
import time

In [ ]:
# install these libraries
%pip install numpy
%pip install pandas

In [ ]:
# external libraries - not installed by default
import numpy as np
import pandas as pd

In [ ]:
# set up our input file path and name
# download from github
in_file_name = 'https://raw.githubusercontent.com/mike-babb/intro_to_python/master/intro_to_python_part_02/words.txt'

In [ ]:
# use pandas to load the data
print('Reading in list of words...')
word_df = pd.read_csv(filepath_or_buffer = in_file_name, sep = ',', header = None)    
# check the first few rows

In [ ]:
# check the first few records
word_df.head()

In [ ]:
# let's specify a a more appropriate column name
col_names = ['word']
word_df.columns = col_names

In [ ]:
# how many words are we working with?
n_words = len(word_df)
print('...found', '{:,}'.format(n_words), 'words to find anagrams for...')

In [ ]:
# convert the only column to a string - just to be safe.
# nan is a word in the dictionary. nan is an internal python value.
word_df['word'] = word_df['word'].astype(np.str)

In [ ]:
# create lower case values of the words
word_df['lcase'] = word_df['word'].str.lower()
# and now drop duplicates
word_df = word_df.drop_duplicates('lcase')

In [ ]:
# 1. find word length
word_df['n_chars'] = word_df['lcase'].str.len()

In [ ]:
# 2. extract the first letter
word_df['first_letter'] = word_df['lcase'].str[:1]

In [ ]:
# 3. Let's aggregate the data to create a new dataframe featuring the counts of words by word length
agg_word_df = word_df['n_chars'].groupby(word_df['n_chars']).agg(np.size).to_frame()
col_names = ['n_words']
agg_word_df.columns = col_names
agg_word_df =  agg_word_df.reset_index()

In [ ]:
agg_word_df.head()
# there are 26 one letter words... that checks out

In [ ]:
# and the tail
agg_word_df.tail()

In [ ]:
# let's do a cross tab - word length by start character
select_columns = ['first_letter', 'n_chars']
ct_word_df = pd.crosstab(index=word_df['first_letter'], columns=word_df['n_chars'])

In [ ]:
# check it
ct_word_df.head()

In [ ]:
#  reset the index and write to excel
ct_word_df = ct_word_df.reset_index()

In [ ]:
ct_word_df.head()

In [ ]:
# the amazing thing about pandas is that we can write to disk in a variety of formats.
# Let's pick excel for now.

In [ ]:
# setup the output path
# this will save the excel file to /content directory.
e_file_name = 'words_analysis.xlsx'

In [ ]:
# create the writer object
e_writer = pd.ExcelWriter(e_file_name)

In [ ]:
# let's first write the list of words
word_df.to_excel(excel_writer=e_writer, sheet_name='word_list', index = False)

In [ ]:
# let's write the count of words by character length
agg_word_df.to_excel(excel_writer=e_writer, sheet_name='word_count_by_length', index = False)

In [ ]:
ct_word_df.to_excel(excel_writer=e_writer, sheet_name='word_count_by_length_by_letter', index = False)

In [ ]:
# save and close the excel file
e_writer.save()
e_writer.close()

# what if we were trying to find all words by a brute-force technique?

In [ ]:
# let's play with pandas to learn some facts about our list of words and show case how to use pandas
# we're going to make extensive use of the string functions in python and the pandas variates
# pure python: https://docs.python.org/3.6/library/string.html
# pandas versions: https://pandas.pydata.org/pandas-docs/stable/text.html#working-with-text-data

In [ ]:
# what is the maximum character length?
max_char_length = agg_word_df['n_chars'].max()

In [ ]:
max_char_length

In [ ]:
# how many possible combinations are there?

In [ ]:
n_permutations = math.factorial(max_char_length)

In [ ]:
n_permutations

In [ ]:
# to check all permutations for one 24-character word
# assuming our computer can perform x checks per second
checks_per_second = 100000

In [ ]:
n_seconds = n_permutations / checks_per_second

In [ ]:
print('examining permutations takes', '{:,}'.format(n_seconds), 'seconds...')

In [ ]:
n_minutes = n_seconds / 60 

In [ ]:
print('examining permutations takes', '{:,}'.format(n_minutes), 'minutes...')

In [ ]:
n_hours = n_minutes / 60

In [ ]:
print('examining permutations takes', '{:,}'.format(n_hours), 'hours...')

In [ ]:
n_days = n_hours / 24

In [ ]:
print('examining permutations takes', '{:,}'.format(n_days), 'days...')

In [ ]:
n_years = n_days / 365

In [ ]:
print('examining permutations takes', '{:,}'.format(n_years), 'years...')

In [ ]:
# if we were to check all permutations of all words...
agg_word_df['n_char_factorial'] = agg_word_df['n_chars'].map(math.factorial)

In [ ]:
# and do that for each word...
agg_word_df['n_char_checks'] = agg_word_df['n_words'] * agg_word_df['n_char_factorial']

In [ ]:
total_checks = agg_word_df['n_char_checks'].sum()

In [ ]:
# assuming we can process:
# 1M permutations a second
checks_per_second = 1000000

In [ ]:
processing_time = total_checks / checks_per_second

In [ ]:
processing_years = processing_time / 60 / 60 / 24 / 365

In [ ]:
print('examining all permutations takes', '{:,}'.format(processing_years), 'years...')

In [ ]:
# that's a long time!

# let's re-think our approach by first focusing on one word: time

In [ ]:
# use python's built-in permutations function from the itertools library to generate
# all possible combinations of letters
# this is 4! which is equal to 4*3*2*1 = 24
for p in permutations('time'):
    print(p)

In [ ]:
type(p)

In [ ]:
# now, let's create words from the tuples
for p in permutations('time'):
    new_word = ''.join(p)
    print(new_word)    

In [ ]:
# and we can sort them. 
for p in permutations('time'):   
    new_word = ''.join(sorted(p))
    print(new_word)

In [ ]:
# so, after sorting, they are all the same.
# let's call this sorted word group the 'eimt' group
sorted_word_group = 'eimt'

In [ ]:
# let's sort a series of words to see if they match.
words_to_examine = ['time', 'mite', 'emit', 'item', 'rite']

In [ ]:
for word in words_to_examine:    
    sorted_word = sorted(word)
    if ''.join(sorted_word) == sorted_word_group:
        print(word, 'is in the', sorted_word_group, 'group')
    else:
        print(word, 'is NOT in the', sorted_word_group, 'group')

In [ ]:
# so we're going to do the same with pandas.
# let's write a function to do this. 
def create_sort_word(word):
    sorted_word = sorted(word)
    output_word = ''.join(sorted_word)    
    return output_word

In [ ]:
# use the map function to apply our custom function and create the sorted word
word_df['word_group'] = word_df['lcase'].map(create_sort_word)

In [ ]:
# we can also do this in seperate steps
word_df['sorted_word'] = word_df['lcase'].map(sorted)

In [ ]:
# or we can combine operations using a lambda function
word_df['word_group_2'] = word_df['sorted_word'].map(lambda x: ''.join(x))

In [ ]:
# we can chain functions together where the output of one function is immediatedly passed to
# another.
word_df['word_group_3'] = word_df['lcase'].map(sorted).map(lambda x: ''.join(x))

In [ ]:
# or we can compactly do it in one line with nested operations
word_df['word_group_4'] = word_df['lcase'].map(lambda x: ''.join(sorted(x)))

In [ ]:
# All of the aboves techniques produce the same results.
# There are multiple ways to accomplish a task. 
# There are tradeoffs between readibilty, code re-use, and speed. 

In [ ]:
# let's check our work
word_df.head()

In [ ]:
# and the tail
word_df.tail()

In [ ]:
# how many sorted word groups have we identified?
# do a simple group by and sort to count our values
word_df['word_group_count'] = 1

In [ ]:
# the columns we want in our output dataframe
select_columns = ['n_chars', 'word_group', 'word_group_count']

In [ ]:
# aggreate the word_df dataframe and count the number of groups.
word_group_df = word_df[select_columns].groupby(select_columns[:-1]).agg(np.size)

In [ ]:
# reset the index    
word_group_df = word_group_df.reset_index()  

In [ ]:
word_group_df.head()

In [ ]:
word_group_df.tail()

In [ ]:
len(word_group_df)

In [ ]:
# what is the largest word group?
word_group_df['word_group_count'].max()

In [ ]:
# and what is that word group?
word_group_df.loc[word_group_df['word_group_count']==10, 'word_group']

In [ ]:
# there are two!

In [ ]:
largest_word_group = word_group_df.loc[word_group_df['word_group_count']==10, 'word_group']

In [ ]:
largest_word_group_df = word_df.loc[word_df['word_group'].isin(largest_word_group), ]

In [ ]:
largest_word_group_df.tail()

In [ ]:
# we've gone from having to compare 234K words with each other to finding 216K word groups
# 14.3K word groups. We've winnowed down the search space dramatically and found a
# way to eliminate unnecessary comparisons.

In [ ]:
word_group_df.head()

In [ ]:
# let's find our anagrams
# select only word groups that have the same number of words in the groups as
# letters in the word.
word_group_df = word_group_df.loc[word_group_df['word_group_count'] > 1, ]

In [ ]:
len(word_df)

In [ ]:
len(word_group_df)

In [ ]:
# there are 14,362 word groups with at least two words in each group.
# we've gone from having to compare 234K words with each other to finding 216K word groups
# and then down to 382 word groups. We've winnowed down the search space dramatically and
# found a way to eliminate unnecessary comparisons.
# So now we pull out the unique word count values, query the df,
# then write to disk.

In [ ]:
n_char_list = word_group_df['n_chars'].unique().tolist()
n_char_list.sort()

In [ ]:
n_char_list[:5]

In [ ]:
word_group_df.head()

In [ ]:
word_df.head()

In [ ]:
# use a hash function to speed up comparisons
# the hash is an integer value of an object:
# python is orders of magnitude faster at comparing integers than strings
word_df['word_hash'] = word_df['word_group'].map(hash)

In [ ]:
word_group_df['word_hash'] = word_group_df['word_group'].map(hash)

In [ ]:
# use the csv writer to write this to disk
output_file_name = 'anagrams_found.txt'
# specify the proper line ending when using a csv writer.
output_file = open(output_file_name, 'w', newline='')
cw = csv.writer(output_file)    
# intialize some counters
n_anagram_groups = 0
n_anagrams = 0

In [ ]:
# start a time to record processing time.
s_time = time.time()

# enumerate over the list of words by character length
for n_char in n_char_list[:2]:
    # subset the word_group_df by focal character length
    curr_df = word_group_df.loc[word_group_df['n_chars']==n_char, ]    
    
    # create a list of word groups where each word group is n_char in length.
    word_group_list = curr_df['word_group'].unique().tolist()
    
    # for use with the computed hash look up
    # word_group_list = curr_df['word_hash'].unique().tolist()
       
    n_word_groups = len(word_group_list)
        
    print('...found', '{:,}'.format(n_word_groups), 'unique word groups within', n_char, 'digit words.')
    for i_wg, wg in enumerate(word_group_list):
        # enumerate() is a function that returns an integer value and the value 
        # in a iterable object.
        
        # get the group of words that correspond to the specific word group
        curr_word_list = word_df.loc[word_df['word_group'] == wg, 'lcase'].tolist()
        
        # using the hash lookup to examine the difference in speed.
        #curr_word_list = word_df.loc[word_df['word_hash'] == wg, 'lcase'].tolist()
        
        # write that word group to the output csv
        cw.writerow(curr_word_list)
        # increment the counters to find the total number of anagram groups
        # and words
        n_anagram_groups += 1
        n_anagrams += len(curr_word_list)

# close the output file
output_file.close()
# mark the time at which processing has ended
e_time = time.time()

In [ ]:
p_time = e_time - s_time

In [ ]:
print('...finding anagrams took', '{:,}'.format(round(p_time,1)), 'seconds...')

In [ ]:
# how many anagram groups and anagrams did we find?
n_anagram_groups = '{:,}'.format(n_anagram_groups)
n_anagrams = '{:,}'.format(n_anagrams)
print('...found', n_anagram_groups, 'anagram groups consisting of',
      n_anagrams, 'words...')